In [1]:
import re
from pathlib import Path

import pandas as pd
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LassoCV
from sklearn.preprocessing import StandardScaler

from run1.lib.classes_ml import DataHandler
from run1.lib.directory import get_directory

In [2]:
CURRENT_DIR = Path.cwd()  # Current directory of the running file
# Get data directory
directory = get_directory(CURRENT_DIR, verbose=True)
DATA_PATH = directory["DATA_PATH"]

Code is running in a Jupyter environment.
ROOT_DIR: c:\Users\admin\Coding\research\weld-ml
DATA_DIR: c:\Users\admin\Coding\research\weld-ml\run1\P03_MF_2\T01_af_features
DATA_PATH: c:\Users\admin\Coding\research\weld-ml\run1\P03_MF_2\T01_af_features\S01_combined_data.xlsx
STUDY_ML_DIR: c:\Users\admin\Coding\research\weld-ml\run1\P03_MF_2\T02_optuna
STUDY_ML_PATH: c:\Users\admin\Coding\research\weld-ml\run1\P03_MF_2\T02_optuna\S02_combine_study.xlsx
STUDY_TABPFN_DIR: c:\Users\admin\Coding\research\weld-ml\run1\P03_MF_2\T11_tabPFN
STUDY_TABPFN_PATH: c:\Users\admin\Coding\research\weld-ml\run1\P03_MF_2\T11_tabPFN\S01_calculate_performance.xlsx


In [3]:
_df = pd.read_excel(DATA_PATH)
print(f"df.shape: {_df.shape}")

df.shape: (378, 61)


In [4]:
# Select columns for features and targets
colsY = [c for c in _df.columns if re.search(r"stress_value", c)]
colsX = [c for c in _df.columns if c not in ["sample_no", "location", *colsY]]

# Select feature columns based on predefined names
colsY = [c for c in colsY if c in ["stress_value_center"]]

_dfY = _df[colsY]
_dfX = _df[colsX]
print("Selected feature columns:", colsX)
print("Selected target columns:", colsY)
print(f"dfX.shape: {_dfX.shape}")
print(f"dfY.shape: {_dfY.shape}")

Selected feature columns: ['position', 'R', 'W', 'D', 'Fx_location', 'Fy_location', 'Fz_location', 'Mz_location', 'Fx__dwell__fft_coefficient__attr_"abs"__coeff_11', 'Fx__dwell__ar_coefficient__coeff_0__k_10', 'Fx__dwell__quantile__q_0.7', 'Fx__dwell__fft_coefficient__attr_"real"__coeff_62', 'Fx__dwell__partial_autocorrelation__lag_6', 'Fx__dwell__change_quantiles__f_agg_"mean"__isabs_True__qh_0.8__ql_0.2', 'Fy__dwell__fft_coefficient__attr_"real"__coeff_71', 'Fy__dwell__fft_coefficient__attr_"abs"__coeff_59', 'Fy__dwell__last_location_of_minimum', 'Fy__dwell__first_location_of_minimum', 'Fy__dwell__change_quantiles__f_agg_"mean"__isabs_True__qh_0.8__ql_0.2', 'Fy__dwell__autocorrelation__lag_3', 'Fz__dwell__fft_coefficient__attr_"angle"__coeff_4', 'Fz__dwell__fft_coefficient__attr_"real"__coeff_4', 'Fz__dwell__fft_coefficient__attr_"angle"__coeff_68', 'Fz__dwell__change_quantiles__f_agg_"mean"__isabs_True__qh_0.4__ql_0.0', 'Fz__dwell__approximate_entropy__m_2__r_0.7', 'Fz__dwell__appro

In [5]:
# %% Extract features and targets
_X = _dfX.values
_Y = _dfY.values
print(f"_X.shape: {_X.shape}")
print(f"_Y.shape: {_Y.shape}")

_X.shape: (378, 56)
_Y.shape: (378, 1)


In [6]:
# Create DataHandler instance
data_handler = DataHandler(
    _X=_X,
    _Y=_Y,
    scalerX=StandardScaler(),
    scalerY=StandardScaler(),
    colsX=colsX,
    colsY=colsY,
)

In [7]:
idx = 1
random_state = 1
test_size = 0.0
data_handler.split_and_scale(random_state=random_state, test_size=test_size)
df_X_train, df_Y_train = data_handler.get_train(as_dataframe=True)
display(df_X_train.head())
display(df_Y_train.head())

No test set, using all data for training.


,position,R,W,D,Fx_location,Fy_location,Fz_location,Mz_location,"Fx__dwell__fft_coefficient__attr_""abs""__coeff_11",Fx__dwell__ar_coefficient__coeff_0__k_10,...,"Fz__weld__fft_coefficient__attr_""real""__coeff_84","Fz__weld__fft_coefficient__attr_""real""__coeff_85",Fz__weld__ratio_beyond_r_sigma__r_1,Fz__weld__ratio_beyond_r_sigma__r_2,Mz__weld__quantile__q_0.1,Mz__weld__energy_ratio_by_chunks__num_segments_10__segment_focus_4,"Mz__weld__fft_coefficient__attr_""angle""__coeff_15",Mz__weld__last_location_of_maximum,"Mz__weld__fft_coefficient__attr_""abs""__coeff_58","Mz__weld__fft_coefficient__attr_""abs""__coeff_83"
0,-5.000000e-01,-1.224745,0.000000,-1.224745,-0.777006,-0.464968,-0.805576,-0.423257,-0.212769,0.165177,...,-0.865086,-0.711205,0.527619,-0.671809,-0.348236,-0.267888,1.361784,-1.218256,0.104113,-0.267514
1,-4.810966e-16,1.224745,1.224745,0.000000,-0.032479,-0.377666,0.320237,-0.355252,-0.525585,-0.384221,...,2.500901,-2.407599,-1.234720,1.252146,-0.315047,-0.390596,0.163715,0.705080,0.052008,-0.431733
2,-1.000000e+00,0.000000,-1.224745,-1.224745,0.064000,-0.782369,-2.024732,-0.465982,0.203409,1.906263,...,1.139794,0.962873,1.948740,-2.009696,-0.536800,-0.613814,-2.594135,0.689238,0.540846,1.248460
3,-5.000000e-01,1.224745,1.224745,1.224745,-0.050435,-0.543303,-0.166273,-0.374813,-0.053910,-0.056061,...,2.072971,-2.038444,0.615044,0.194399,-0.294181,-0.030634,0.100716,0.825404,0.194849,-0.038822
4,-1.000000e+00,1.224745,0.000000,-1.224745,-0.117693,-0.393217,-1.237342,-0.468396,-0.531406,-0.292844,...,-0.271261,0.027573,0.606539,-0.316606,-0.318095,-0.745745,-0.064711,0.844101,1.326479,-0.028616


,stress_value_center
0,-1.088213
1,0.678466
2,-0.499320
3,-0.057650
4,-0.646543


In [8]:
# Train LassoCV model
model = LassoCV(cv=5, random_state=0, max_iter=10000).fit(
    df_X_train, df_Y_train.values.ravel()
)

In [9]:
# Rank features from coefficients
selector = SelectFromModel(model, prefit=True, threshold="mean")
feature_idx = selector.get_support()
feature_names = df_X_train.columns[feature_idx]
feature_importance = abs(model.coef_[feature_idx])
feature_ranking = pd.DataFrame(
    {"Feature": feature_names, "Importance": feature_importance}
).sort_values(by="Importance", ascending=False) 
feature_ranking.reset_index(drop=True, inplace=True)
display(feature_ranking)

,Feature,Importance
0,position,0.484863
1,Fz__weld__ratio_beyond_r_sigma__r_1,0.215905
2,"Fx__weld__fft_coefficient__attr_""abs""__coeff_58",0.087760
3,Fx_location,0.086970
4,"Fy__weld__fft_coefficient__attr_""real""__coeff_51",0.086441
5,Fy__weld__ar_coefficient__coeff_5__k_10,0.068817
6,"Fz__weld__fft_coefficient__attr_""real""__coeff_84",0.065429
7,"Mz__dwell__fft_coefficient__attr_""real""__coeff_42",0.053206
8,Fy__dwell__autocorrelation__lag_3,0.047136
9,"Mz__weld__fft_coefficient__attr_""angle""__coeff_15",0.046982


In [10]:
# Feature selection using Lasso coefficients
# Use a small threshold to capture exactly zero coefficients

sel = SelectFromModel(model, prefit=True, threshold=1e-2)

# To get the names of selected features (if X_train is a pandas DataFrame)
selected_features = df_X_train.columns[sel.get_support()]

print(f"Selected features: {list(selected_features)}")

Selected features: ['position', 'W', 'Fx_location', 'Fx__dwell__partial_autocorrelation__lag_6', 'Fy__dwell__fft_coefficient__attr_"real"__coeff_71', 'Fy__dwell__autocorrelation__lag_3', 'Mz__dwell__augmented_dickey_fuller__attr_"teststat"__autolag_"AIC"', 'Mz__dwell__fft_coefficient__attr_"real"__coeff_42', 'Fx__weld__change_quantiles__f_agg_"var"__isabs_True__qh_0.6__ql_0.2', 'Fx__weld__fft_coefficient__attr_"abs"__coeff_5', 'Fx__weld__fft_coefficient__attr_"abs"__coeff_58', 'Fy__weld__fft_coefficient__attr_"imag"__coeff_61', 'Fy__weld__fft_coefficient__attr_"real"__coeff_51', 'Fy__weld__ar_coefficient__coeff_5__k_10', 'Fz__weld__fft_coefficient__attr_"real"__coeff_84', 'Fz__weld__ratio_beyond_r_sigma__r_1', 'Mz__weld__fft_coefficient__attr_"angle"__coeff_15']
